In [169]:
from langchain_community.document_loaders import Docx2txtLoader
from transformers import pipeline
import re

### 1 - NER using open soure (hugging face) model

In [1]:
#Lire un fichier docx et extraire son texte.
def extract_text_from_docx(docx_path):
    data  = Docx2txtLoader(docx_path).load()
    return data[0].page_content.strip()

#Génèrer les regex à des noms d'entités qu'on souhaite extraire
def format_entity_patterns(entity_names):
    return [rf"{re.escape(name)}\s*\n(.+)" for name in entity_names]


#Appliquer les regex pour extraire des entités nommées
def extract_entities(text,
                      entities_names =  ["Counterparty", "Initial Valuation Date", "Notional", "Valuation Date", "Maturity", "Underlying", "Coupon", "Barrier", "Calendar"], 
                      entities_to_extract = ["Party A", "Initial Valuation Date", "Notional Amount (N)", "Valuation Date", "Termination Date",  "Underlying", "Coupon (C)", "Barrier (B)", "Business Day"]):
    entities = {}
    patterns = dict(zip(entities_names,  format_entity_patterns(entities_to_extract)))
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        entities[key] = match.group(1) if match else None
    return entities

In [171]:
docx_text = extract_text_from_docx("data/ZF4894_ALV_07Aug2026_physical.docx")

extract_entities(docx_text)

{'Counterparty': 'BANK ABC',
 'Initial Valuation Date': '31 January 2025',
 'Notional': 'EUR 1 million',
 'Valuation Date': '31 January 2025',
 'Maturity': '07 August 2026',
 'Underlying': 'Allianz SE (ISIN DE0008404005, Reuters: ALVG.DE)',
 'Coupon': '0%',
 'Barrier': '75.00% of Shareini  ',
 'Calendar': 'TARGET '}

### 2 - NER using open soure (hugging face) model

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [174]:
def read_text_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()
    
def model_loader(model_path) :
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    return pipeline("ner", model=model, tokenizer=tokenizer,  aggregation_strategy="first")


def extract_named_entities_from_text(ner_pipeline, text) :
    results = ner_pipeline(text)
    entities = [{k: d[k] for k in ["entity_group", "score", "word"]} for d in results]
    return entities

In [ ]:
model_path = "./pretrained-models/distilbert-NER"
ner_pipeline = model_loader(model_path)

text = read_text_file("data/FR001400QV82_AVMAFC_30Jun2028.txt")
extract_named_entities_from_text(ner_pipeline, text)

NameError: name 'model_loader' is not defined

### NER for Verbose and complex PDF using RAG approach

In [6]:
from src.llm_and_rag_ner import extract_named_entities

ModuleNotFoundError: No module named 'ollama'

In [ ]:


model='llama3.2'
text = "Elon Musk a fondé SpaceX à Los Angeles. Apple et Google investissent en France."

# Extraction des entités
entities = extract_named_entities(model, text)

# Affichage des entités extraites
print(json.dumps(entities, indent=2, ensure_ascii=False))


ModuleNotFoundError: No module named 'ollama'

In [3]:
import ollama

ModuleNotFoundError: No module named 'ollama'